In [7]:
import pretty_midi
import numpy as np
from pathlib import Path

#INPUT_MIDI = Path("../data/input_midi/example_01.mid")
INPUT_MIDI = Path("../data/input_midi/max_piano.mid")
#OUTPUT_MIDI = Path("../data/output_midi/example_01_ai.mid")
OUTPUT_MIDI = Path("../data/output_midi/max_piano_suite.mid")

TEMPO = 120
TIME_SIG = (4, 4)


In [8]:
midi = pretty_midi.PrettyMIDI(str(INPUT_MIDI))
instrument = midi.instruments[0]
notes = instrument.notes

print(f"Notes: {len(notes)}")
print(f"Pitch range: {min(n.pitch for n in notes)}–{max(n.pitch for n in notes)}")
print(f"Duration: {midi.get_end_time():.2f}s")

Notes: 219
Pitch range: 36–100
Duration: 8.13s


In [9]:
pitches = [n.pitch for n in notes]
avg_pitch = np.mean(pitches)
num_notes = len(notes)

seconds_per_bar = 60 / TEMPO * TIME_SIG[0]
bars = int(round(midi.get_end_time() / seconds_per_bar))

def density_bucket(n):
    if n < 20:
        return "low"
    elif n < 60:
        return "medium"
    return "high"

def register_bucket(p):
    if p < 50:
        return "low"
    elif p < 70:
        return "mid"
    return "high"

features = {
    "density": density_bucket(num_notes),
    "register": register_bucket(avg_pitch),
    "bars": bars,
}

features

{'density': 'high', 'register': 'mid', 'bars': 4}

In [10]:
def decide_action(features):
    if features["density"] == "high":
        return {
            "action": "play",
            "role": "counter_melody",
            "density": "low",
            "register": "lower",
            "bars": features["bars"],
        }
    return {
        "action": "play",
        "role": "counter_melody",
        "density": "medium",
        "register": "same",
        "bars": features["bars"],
    }

action = decide_action(features)
action

{'action': 'play',
 'role': 'counter_melody',
 'density': 'low',
 'register': 'lower',
 'bars': 4}

In [11]:
def generate_response(action, start_time):
    response = pretty_midi.PrettyMIDI()
    inst = pretty_midi.Instrument(
        program=pretty_midi.instrument_name_to_program("Electric Piano 1")
    )

    # simple D minor tones for MVP
    scale = [62, 65, 69]
    bar_len = 60 / TEMPO * TIME_SIG[0]

    t = start_time
    notes_per_bar = 1 if action["density"] == "low" else 2

    for _ in range(action["bars"]):
        for i in range(notes_per_bar):
            pitch = np.random.choice(scale)
            note = pretty_midi.Note(
                velocity=70 - i * 10,
                pitch=pitch,
                start=t + i * 0.5,
                end=t + i * 0.5 + 0.4,
            )
            inst.notes.append(note)
        t += bar_len

    response.instruments.append(inst)
    return response

In [12]:
def generate_motivic_response(notes, start_time, tempo=120, bars=2):
    """
    Generate a motivic response based on the last phrase.

    Combines:
    - exact repetition
    - fragmentation
    - transposed sequencing
    """

    if len(notes) < 3:
        return None  # not enough material

    # Sort by time
    notes = sorted(notes, key=lambda n: n.start)

    # Extract last motif (last 4 notes max)
    motif_notes = notes[-4:]
    motif_pitches = [n.pitch for n in motif_notes]

    # Choose response strategy
    strategy = np.random.choice(
        ["repeat", "fragment", "sequence"]
    )

    # -------------------------
    # STRATEGY 1 — Exact repeat
    # -------------------------
    if strategy == "repeat":
        new_pitches = motif_pitches

    # -------------------------
    # STRATEGY 2 — Fragmentation
    # -------------------------
    elif strategy == "fragment":
        fragment_size = np.random.choice([2, 3])
        fragment = motif_pitches[-fragment_size:]
        repeats = np.random.choice([2, 3])
        new_pitches = fragment * repeats

    # -------------------------
    # STRATEGY 3 — Transposed sequence
    # -------------------------
    else:
        intervals = np.diff(motif_pitches)
        fragment_size = min(3, len(intervals))
        base_fragment = motif_pitches[:fragment_size]

        shift = np.random.choice([2, -2, 3])  # stepwise or small leap
        new_pitches = []

        for i in range(2):
            transposed = [p + i * shift for p in base_fragment]
            new_pitches.extend(transposed)

    # Clamp register (stay tasteful)
    new_pitches = [max(48, min(84, p)) for p in new_pitches]

    # -------------------------
    # Create MIDI
    # -------------------------
    response = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(
        program=pretty_midi.instrument_name_to_program("Electric Piano 1")
    )

    seconds_per_beat = 60 / tempo
    note_duration = seconds_per_beat * 0.9

    t = start_time

    for pitch in new_pitches:
        note = pretty_midi.Note(
            velocity=80,
            pitch=int(pitch),
            start=t,
            end=t + note_duration
        )
        instrument.notes.append(note)
        t += seconds_per_beat

    response.instruments.append(instrument)

    return response

In [13]:
start_time = midi.get_end_time()

response_midi = generate_motivic_response(
    notes=instrument.notes,
    start_time=start_time,
    tempo=TEMPO,
    bars=2
)

combined = pretty_midi.PrettyMIDI()
combined.instruments = midi.instruments + response_midi.instruments

combined.write(str(OUTPUT_MIDI))